# LAB | Feature Engineering

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In [ ]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

**Check the shape of your data**

In [ ]:
spaceship.shape

**Check for data types**

In [ ]:
spaceship.dtypes

**Check for missing values**

In [ ]:
spaceship.isna().sum()

There are multiple strategies to handle missing data

- Removing all rows or all columns containing missing data.
- Filling all missing values with a value (mean in continouos or mode in categorical for example).
- Filling all missing values with an algorithm.

For this exercise, because we have such low amount of null values, we will drop rows containing any missing value. 

In [ ]:
spaceship = spaceship.dropna()
spaceship = spaceship.reset_index(drop=True)

- **Cabin** is too granular - transform it in order to obtain {'A', 'B', 'C', 'D', 'E', 'F', 'G', 'T'}

In [ ]:
spaceship['Cabin'] = spaceship['Cabin'].apply(lambda x: x[0])

In [ ]:
spaceship.Cabin.value_counts()

- Drop PassengerId and Name

In [ ]:
spaceship = spaceship.drop(["PassengerId", "Name"], axis=1)
spaceship

- For non-numerical columns, do dummies.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False)

ohe.fit(spaceship[['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP']])
spaceship_ohe = ohe.transform(spaceship[['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP']])
spaceship_ohe

In [ ]:
spaceship_ohe_df = pd.DataFrame(spaceship_ohe, columns=ohe.get_feature_names_out(), index=spaceship.index)
spaceship_ohe_df

In [ ]:
spaceship_num = spaceship.select_dtypes(include=['number'])
spaceship_fin = pd.concat([spaceship_num, spaceship_ohe_df], axis = 1)
target = spaceship.Transported

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler

normalizer = MinMaxScaler()
normalizer.fit(spaceship_fin)

In [ ]:
spaceship_norm = normalizer.transform(spaceship_fin)
spaceship_norm = pd.DataFrame(spaceship_norm, columns=spaceship_fin.columns, index=spaceship_fin.index )
spaceship_norm.head()

**Perform Train Test Split**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(spaceship_norm, target, test_size = 0.20, random_state=0)

**Model Selection**

In this exercise we will be using **KNN** as our predictive model.

In [ ]:
knn = KNeighborsRegressor()
knn.fit(X_train, y_train)

- Evaluate your model's performance. Comment it

In [ ]:
print(f"The R2 of the model on the TEST set is: {knn.score(X_test, y_test): .2f}")

In [ ]:
print("R2 of 0.32 indicates that the model is not working reliably. An option would be to repeat the test with standardization instead of normalization and see if the results improve.")

In [ ]:
scaler = StandardScaler()
scaler.fit(spaceship_fin)

In [ ]:
spaceship_stand = scaler.transform(spaceship_fin)
spaceship_stand = pd.DataFrame(spaceship_stand, columns=spaceship_fin.columns, index=spaceship_fin.index)
spaceship_stand.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(spaceship_stand, target, test_size = 0.20, random_state=0)

In [ ]:
knn.fit(X_train, y_train)

In [ ]:
print(f"The R2 of the model on the TEST set is: {knn.score(X_test, y_test): .2f}")

In [ ]:
print("After standardizing the data, the model improved slightly but isn't reliable enough in order to work as expected.")